In [2]:
import os

In [3]:
%pwd

'd:\\coding\\ml dl , data science\\NLP-project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\coding\\ml dl , data science\\NLP-project'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config


In [13]:
!pip install torch --upgrade



                                              0.0/203.1 MB ? eta -:--:--
                                            0.0/203.1 MB 660.6 kB/s eta 0:05:08
                                            0.1/203.1 MB 653.6 kB/s eta 0:05:11
                                            0.1/203.1 MB 798.9 kB/s eta 0:04:15
                                              0.2/203.1 MB 1.2 MB/s eta 0:02:50
                                              0.2/203.1 MB 1.2 MB/s eta 0:02:50
                                            0.3/203.1 MB 911.0 kB/s eta 0:03:43
                                              0.4/203.1 MB 1.2 MB/s eta 0:02:53
                                              0.5/203.1 MB 1.2 MB/s eta 0:02:49
                                              0.6/203.1 MB 1.5 MB/s eta 0:02:14
                                              0.7/203.1 MB 1.6 MB/s eta 0:02:10
                                              0.9/203.1 MB 1.8 MB/s eta 0:01:54
                                              1

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'D:\\coding\\ml dl , data science\\NLP-project\\textS\\Lib\\site-packages\\~orch\\lib\\asmjit.dll'
Check the permissions.


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [10]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [11]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-11-13 14:45:49,165: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-13 14:45:49,172: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-13 14:45:49,173: INFO: common: created directory at: artifacts]
[2024-11-13 14:45:49,174: INFO: common: created directory at: artifacts/data_transformation]


d:\coding\ml dl , data science\NLP-project\textS\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\RAJVARDHAN PATIL\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
d:\coding\ml dl , data science\NLP-project\textS\Lib\site-packages\transform